In [6]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, split, explode, format_number

base_path = r'G:\.shortcut-targets-by-id\1eDKYURr-Qm222Ul6PoZLc4b99cuwHTC3\Cường\Đại học\DS200\Lab 1'

spark = SparkSession.builder \
    .appName("DS200 Lab1 - Bai 2") \
    .getOrCreate()

try:
    movies_df = spark.read.format("csv") \
        .option("header", "false") \
        .option("inferSchema", "true") \
        .load(os.path.join(base_path, "movies.cleaned.txt")) \
        .toDF("MovieID", "Title", "Genres")

    ratings_temp_df = spark.read.format("csv") \
        .option("header", "false") \
        .load(os.path.join(base_path, "ratings.cleaned.txt")) \
        .toDF("UserID_str", "MovieID_str", "Rating_str", "Timestamp_str")

    ratings_df = ratings_temp_df.select(
        col("UserID_str").cast("int").alias("UserID"),
        col("MovieID_str").cast("int").alias("MovieID"),
        col("Rating_str").cast("float").alias("Rating"),
        col("Timestamp_str").cast("long").alias("Timestamp")
    )

    ratings_df.show(5)

except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    spark.stop()
    exit()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|   537|   1043|   4.0|964982703|
|   642|   3791|   3.5|964982710|
|   789|   4862|   5.0|964982720|
|   556|   5930|   4.5|964982730|
|   612|   5274|   3.0|964982740|
+------+-------+------+---------+
only showing top 5 rows


# Tách các thể loại thành các hàng riêng biệt

In [ ]:
# Join ratings_df với movies_df để lấy cột Genres
ratings_with_genres_df = ratings_df.join(movies_df, "MovieID")

# Tách cột 'Genres' bằng dấu '|' và explode mảng kết quả thành các hàng riêng biệt
exploded_genres_df = ratings_with_genres_df.withColumn(
    "Genre",
    explode(split(col("Genres"), "\|"))  
)

print("\nKiểm tra dữ liệu sau khi đã tách thể loại:")
exploded_genres_df.select("Title", "Rating", "Genre").show(10)


Kiểm tra dữ liệu sau khi đã tách thể loại:
+------------------+------+----------+
|             Title|Rating|     Genre|
+------------------+------+----------+
|  Toy Story (1995)|   4.5| Animation|
|  Toy Story (1995)|   4.5|  Children|
|  Toy Story (1995)|   4.5|    Comedy|
|  Toy Story (1995)|   4.0| Animation|
|  Toy Story (1995)|   4.0|  Children|
|  Toy Story (1995)|   4.0|    Comedy|
| The Matrix (1999)|   4.5|    Action|
| The Matrix (1999)|   4.5|    Sci-Fi|
| The Matrix (1999)|   3.0|    Action|
| The Matrix (1999)|   3.0|    Sci-Fi|
+------------------+------+----------+
only showing top 10 rows


# Phân Tích Đánh Giá Theo Thể Loại

In [4]:
genre_stats_df = exploded_genres_df.groupBy("Genre").agg(
    avg("Rating").alias("AverageRating"),
    count("Rating").alias("TotalRatings")
)

genre_stats_df.select(
    col("Genre"),
    format_number(col("AverageRating"), 2).alias("Avg"),
    col("TotalRatings").alias("Count")
).orderBy("Genre").show()

+----------+----+-----+
|     Genre| Avg|Count|
+----------+----+-----+
|    Action|3.73|   20|
| Adventure|3.83|   12|
| Animation|4.17|    6|
| Biography|4.14|   14|
|    Comedy|3.67|    6|
|     Crime|3.68|   14|
|     Drama|3.88|   26|
|   Mystery|4.00|    2|
| Adventure|3.86|   18|
|  Children|4.25|    2|
|    Comedy|4.25|    6|
|     Crime|4.00|    6|
|     Drama|3.77|   50|
|    Family|4.25|    2|
|   Fantasy|3.95|   10|
|   History|4.00|    6|
|     Music|3.88|    4|
|   Mystery|4.00|    6|
|   Romance|3.75|    8|
|    Sci-Fi|3.85|   20|
+----------+----+-----+
only showing top 20 rows


In [5]:
spark.stop()